In [ ]:
# Loading Libraries and Visualizing Data
import os
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Dense, Bidirectional
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from ta.momentum import RSIIndicator

random.seed(7)
tf.random.set_seed(7)
np.random.seed(7)
tf.compat.v1.set_random_seed(7)

os.chdir("C:/Programming/Stock prediction") #set your own working directory

# load & preprocess data function
def load_stock_data(filename="AAPL.csv"):
    df = pd.read_csv(filename)
    df['date'] = pd.to_datetime(df['date'])
    df['log_return'] = np.log(df['close'] / df['close'].shift(1))
    df['volatility_5d'] = np.log(df['close'] / df['close'].shift(1)).rolling(window=5).std()
    df['volatility_21d'] = np.log(df['close'] / df['close'].shift(1)).rolling(window=21).std()
    df = df.dropna().reset_index(drop=True)
    return df

def plot_returns_and_volatility(df):
    plt.figure(figsize=(12, 6))
    plt.plot(df['date'], df['log_return'], label="Daily Log Return", color='skyblue', alpha=0.7)
    plt.title("Apple Daily Log Returns")
    plt.xlabel("Date")
    plt.ylabel("Log Return")
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    plt.show()

    plt.figure(figsize=(12, 6))
    plt.plot(df['date'], df['volatility_5d'], label="5-Day Volatility", color='blue', alpha=0.8)
    plt.plot(df['date'], df['volatility_21d'], label="21-Day Volatility",color='orange', alpha=0.8)
    plt.title("Realized Volatility (5-day vs 21-day)")
    plt.xlabel("Date")
    plt.ylabel("Volatility")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

df = load_stock_data("AAPL.csv")
plot_returns_and_volatility(df)


In [ ]:
# Model building and training
# feature engineering function
def add_features(df):
    df = df.copy()
    df['log_return'] = np.log(df['close'] / df['close'].shift(1))
    df['lag1'] = df['log_return'].shift(1)
    df['lag2'] = df['log_return'].shift(2)
    df['r2'] = df['log_return']**2
    df['z_log_return'] = (df['log_return'] - df['log_return'].rolling(21).mean()) / df['log_return'].rolling(21).std()
    df['price_shock'] = (df['log_return'].abs() > 2 * df['log_return'].rolling(21).std()).astype(int)
    df['rsi'] = RSIIndicator(close=df['close'], window=14).rsi()
    df['log_volume_change'] = np.log(df['volume'] / df['volume'].shift(1))
    df['log_range'] = np.log(df['high'] / df['low'])
    df['vol_5'] = df['log_return'].rolling(window=5).std()
    df['volatility_7'] = df['log_return'].rolling(window=7).std()
    df['volatility_21'] = df['log_return'].rolling(window=21).std()
    df['future_vol'] = df['log_return'].rolling(window=5).std().shift(-5)    
    return df.dropna().reset_index(drop=True)

data = add_features(df)

# sequence generator
def create_sequences(train_x_scaled, train_y_scaled, seq_length):
    train_seq_x, train_seq_y = [], []
    for i in range(seq_length, len(train_x_scaled)):
        train_seq_x.append(train_x_scaled[i - seq_length:i])
        train_seq_y.append(train_y_scaled[i])
    return np.array(train_seq_x), np.array(train_seq_y)

# build model
def build_lstm_model(input_shape):
    model = Sequential([
        Bidirectional(LSTM(80, return_sequences=True, input_shape=input_shape)),
        Dropout(0.2),
        LSTM(100, return_sequences=True),
        Dropout(0.2),
        LSTM(120),
        Dropout(0.3),
        Dense(32),
        Dense(1)])
    return model

# compile model
def compile_model(model):
    model.compile(optimizer='adam',
                  loss='mean_squared_error',
                  metrics=[RootMeanSquaredError()])
    return model

# load and process dataset
seq_length = 45
x = ['log_return','lag1', 'lag2','r2',
    'z_log_return', 'price_shock','rsi',
    'log_volume_change', 'log_range',
    'vol_5', 'volatility_7', 'volatility_21']
y = 'future_vol'
    
# split data
num_samples = len(data)
train_cutoff = int(num_samples * 0.6)
test_cutoff = int(num_samples * 0.8)

train_data = data.iloc[:train_cutoff]
test_data = data.iloc[train_cutoff:test_cutoff]
out_of_sample_data = data.iloc[test_cutoff:]

train_y = train_data[[y]].values
test_y = test_data[[y]].values

# scale features and target
x_scaler = MinMaxScaler()
y_scaler = MinMaxScaler()

train_x_scaled = x_scaler.fit_transform(train_data[x])
test_x_scaled = x_scaler.transform(test_data[x])
train_y_scaled = y_scaler.fit_transform(train_y)
test_y_scaled = y_scaler.transform(test_y)

# create training sequences
train_seq_x, train_seq_y = create_sequences(train_x_scaled,
                                            train_y_scaled,
                                            seq_length)
train_seq_x = train_seq_x.reshape(-1, seq_length, len(x))

# model training
input_shape = (seq_length, len(x))
model = build_lstm_model(input_shape)
model = compile_model(model)

early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, verbose=1)

history = model.fit(
    train_seq_x, train_seq_y,
    epochs=100,
    batch_size=32,
    validation_split=0.1,
    callbacks=[early_stop, reduce_lr],
    verbose=1)

model.save('bilstm_volatility_model.keras')

# plot training history
plt.figure(figsize=(10, 5))
plt.plot(history.history['loss'], label='Training Loss', linewidth=2)
plt.plot(history.history['val_loss'], label='Validation Loss', linestyle='--', linewidth=2)
plt.title('Training vs. Validation Loss (Volatility Forecasting)')
plt.xlabel('Epoch')
plt.ylabel('Loss (MSE)')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# Model Testing and Prediction
def generate_test_sequences(train_x_scaled,
                            test_x_scaled,
                            test_y_scaled,
                            seq_length):
    combined = np.concatenate([train_x_scaled[-seq_length:], test_x_scaled],
                              axis=0)
 
    test_seq_x, test_seq_y = [], []
    for i in range(seq_length, len(combined)):
        test_seq_x.append(combined[i - seq_length:i])
        test_seq_y.append(test_y_scaled[i - seq_length])    
    return np.array(test_seq_x), np.array(test_seq_y) 

# create sequences for test data
scaled_test_seq_x, scaled_test_seq_y = generate_test_sequences(
    train_x_scaled,
    test_x_scaled,
    test_y_scaled,
    seq_length)

# prediction
y_pred_scaled = model.predict(scaled_test_seq_x)

# inverse transform to original scale
y_pred_actual = y_scaler.inverse_transform(y_pred_scaled)
y_test_actual = y_scaler.inverse_transform(scaled_test_seq_y.reshape(-1, 1))

# plotting function
def plot_forecast_vs_actual(y_test_actual, y_pred_actual, date_series):
    plt.figure(figsize=(12, 6))
    plt.plot(date_series, y_test_actual, label="Actual Volatility", color='blue')
    plt.plot(date_series, y_pred_actual, label="Predicted Volatility", color='red')
    plt.title("BiLSTM Forecast vs. Actual (Test Set)")
    plt.xlabel("Date")
    plt.ylabel("Volatility")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

# align dates
aligned_test_dates = test_data['date'].iloc[-len(y_test_actual):].reset_index(drop=True)

# plot the forecast
plot_forecast_vs_actual(y_test_actual.flatten(), y_pred_actual.flatten(), aligned_test_dates)

mae, mse = mean_absolute_error(y_test_actual, y_pred_actual), mean_squared_error(y_test_actual, y_pred_actual)
rmse = np.sqrt(mse)
print(f"BiLSTM - MAE: {mae:.6f} | MSE: {mse:.6f} | RMSE: {rmse:.6f}")



In [ ]:
# Out of Sample Forecasting
def forecast_out_of_sample(model,
                           test_x_scaled,
                           oos_x_scaled,
                           oos_y_scaled,
                           seq_length,
                           y_scaler):
    combined_input = np.concatenate([test_x_scaled[-seq_length:], oos_x_scaled],
                                    axis=0)

    oos_seq_x, oos_seq_y = [], []
    for i in range(seq_length, len(combined_input)):
        oos_seq_x.append(combined_input[i - seq_length:i])
        oos_seq_y.append(oos_y_scaled[i - seq_length])

    oos_seq_x = np.array(oos_seq_x)
    oos_seq_y = np.array(oos_seq_y)
    scaled_oos_pred = model.predict(oos_seq_x)
    oos_pred = y_scaler.inverse_transform(scaled_oos_pred)
    oos_actual = y_scaler.inverse_transform(oos_seq_y.reshape(-1, 1))
    return oos_actual, oos_pred

# scale out of sample features and target
oos_x_scaled = x_scaler.transform(out_of_sample_data[x])
oos_y = out_of_sample_data[[y]].values
oos_y_scaled = y_scaler.transform(oos_y)

# forecast
y_oos_actual, y_oos_pred = forecast_out_of_sample(
    model,
    test_x_scaled,
    oos_x_scaled,
    oos_y_scaled,
    seq_length,
    y_scaler)

# plot forecast
dates_oos = out_of_sample_data['date'].iloc[seq_length:].reset_index(drop=True)

y_oos_actual = y_oos_actual[:len(dates_oos)]  
y_oos_pred = y_oos_pred[:len(dates_oos)]     

plt.figure(figsize=(12, 6))
plt.plot(dates_oos, y_oos_actual, label='Actual Volatility', color='blue')
plt.plot(dates_oos, y_oos_pred, label='Predicted Volatility', color='red')
plt.title('BiLSTM Out-of-Sample Forecast')
plt.xlabel('Date')
plt.ylabel('Volatility')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

mae_, mse_ = mean_absolute_error(y_oos_actual, y_oos_pred), mean_squared_error(y_oos_actual, y_oos_pred)
rmse_ = np.sqrt(mse_)
print(f"BiLSTM - MAE: {mae_:.6f} | MSE: {mse_:.6f} | RMSE: {rmse_:.6f}")


In [ ]:
# Test metrics
mae, mse = mean_absolute_error(y_test_actual, y_pred_actual), mean_squared_error(y_test_actual, y_pred_actual)
rmse = np.sqrt(mse)
test_metrics = f"Test Metrics - MAE: {mae:.6f} | MSE: {mse:.6f} | RMSE: {rmse:.6f}"

# Out-of-sample metrics
mae_, mse_ = mean_absolute_error(y_oos_actual, y_oos_pred), mean_squared_error(y_oos_actual, y_oos_pred)
rmse_ = np.sqrt(mse_)
oos_metrics = f"Out-of-Sample Metrics - MAE: {mae_:.6f} | MSE: {mse_:.6f} | RMSE: {rmse_:.6f}"

# Save to text files
with open('test_metrics_bilstm.txt', 'w') as f:
    f.write(test_metrics)

with open('oos_metrics_bilstm.txt', 'w') as f:
    f.write(oos_metrics)